LORA Layer

In [2]:
import torch


class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.W_a = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.W_b = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.W_a @ self.W_b)
        return x


class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

ModuleNotFoundError: No module named 'torch'

In [ ]:
torch.manual_seed(123)

# a simple linear layer with 10 inputs and 1 output
# requires_grad=False makes it non-trainable
linear_layer = torch.nn.Linear(10, 1)

# a simple example input
x = torch.rand((1, 10))

linear_layer(x)

tensor([[-0.5745]], grad_fn=<AddmmBackward0>)

Replace linear layer with LoRA layer:

In [ ]:
lora_layer = LinearWithLoRA(linear=linear_layer, rank=8, alpha=1)
lora_layer(x)

tensor([[-0.5745]], grad_fn=<AddBackward0>)

Note that the LoRA layer will not change the linear layer output until it's trained because its W_b weight matrix is initialized to all zeros. For LoRA to take effect, we have to train the model. Below is a simple toy example.

Let's simulate a simple weight update:

In [ ]:
lora_layer.lora.W_b = torch.nn.Parameter(lora_layer.lora.W_b + 0.01 * x[0])

We can now see that the output has changed:

In [ ]:
lora_layer(x)

tensor([[-0.5863, -0.5758, -0.5779, -0.5800, -0.5814, -0.5766, -0.5887, -0.5811,
         -0.5859, -0.5892]], grad_fn=<AddBackward0>)

Local Dataset Utilities

In [ ]:
#COMMENT

!pip install datasets
!pip install lightning

In [ ]:
import os
import os.path as op
import sys
import tarfile
import time

from datasets import load_dataset
import numpy as np
import pandas as pd
from packaging import version
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import urllib


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.0**2 * duration)
    percent = count * block_size * 100.0 / total_size

    sys.stdout.write(
        f"\r{int(percent)}% | {progress_size / (1024.**2):.2f} MB "
        f"| {speed:.2f} MB/s | {duration:.2f} sec elapsed"
    )
    sys.stdout.flush()


def download_dataset():
    source = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    target = "aclImdb_v1.tar.gz"

    if os.path.exists(target):
        os.remove(target)

    if not os.path.isdir("aclImdb") and not os.path.isfile("aclImdb_v1.tar.gz"):
        urllib.request.urlretrieve(source, target, reporthook)

    if not os.path.isdir("aclImdb"):

        with tarfile.open(target, "r:gz") as tar:
            tar.extractall()


def load_dataset_into_to_dataframe():
    basepath = "aclImdb"

    labels = {"pos": 1, "neg": 0}

    df = pd.DataFrame()

    with tqdm(total=50000) as pbar:
        for s in ("test", "train"):
            for l in ("pos", "neg"):
                path = os.path.join(basepath, s, l)
                for file in sorted(os.listdir(path)):
                    with open(os.path.join(path, file), "r", encoding="utf-8") as infile:
                        txt = infile.read()

                    if version.parse(pd.__version__) >= version.parse("1.3.2"):
                        x = pd.DataFrame(
                            [[txt, labels[l]]], columns=["review", "sentiment"]
                        )
                        df = pd.concat([df, x], ignore_index=False)

                    else:
                        df = df.append([[txt, labels[l]]], ignore_index=True)
                    pbar.update()
    df.columns = ["text", "label"]

    np.random.seed(0)
    df = df.reindex(np.random.permutation(df.index))

    print("Class distribution:")
    np.bincount(df["label"].values)

    return df


def partition_dataset(df):
    df_shuffled = df.sample(frac=1, random_state=1).reset_index()

    df_train = df_shuffled.iloc[:35_000]
    df_val = df_shuffled.iloc[35_000:40_000]
    df_test = df_shuffled.iloc[40_000:]

    if not op.exists("data"):
        os.makedirs("data")
    df_train.to_csv(op.join("data", "train.csv"), index=False, encoding="utf-8")
    df_val.to_csv(op.join("data", "val.csv"), index=False, encoding="utf-8")
    df_test.to_csv(op.join("data", "test.csv"), index=False, encoding="utf-8")


class IMDBDataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows


def get_dataset():
    files = ("test.csv", "train.csv", "val.csv")
    download = True

    for f in files:
        if not os.path.exists(f):
            download = False

    if download is False:
        download_dataset()
        df = load_dataset_into_to_dataframe()
        partition_dataset(df)

    df_train = pd.read_csv(op.join("data", "train.csv"))
    df_val = pd.read_csv(op.join("data", "val.csv"))
    df_test = pd.read_csv(op.join("data", "test.csv"))

    return df_train, df_val, df_test


def tokenization():
    imdb_dataset = load_dataset(
        "csv",
        data_files={
            "train": op.join("data", "train.csv"),
            "validation": op.join("data", "val.csv"),
            "test": op.join("data", "test.csv"),
        },
    )
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_text(batch):
        return tokenizer(batch["text"], truncation=True, padding=True)

    imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)
    imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    return imdb_tokenized


def setup_dataloaders(imdb_tokenized):
    train_dataset = IMDBDataset(imdb_tokenized, partition_key="train")
    val_dataset = IMDBDataset(imdb_tokenized, partition_key="validation")
    test_dataset = IMDBDataset(imdb_tokenized, partition_key="test")

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=12,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=12,
        num_workers=4ß
    )
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=12,
        num_workers=4
    )
    return train_loader, val_loader, test_loader

Local Model Utilities

In [ ]:
import time

import lightning as L
import torchmetrics
import torch
import torch.nn.functional as F


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100


def train(num_epochs, model, optimizer, train_loader, device):

    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):

            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)

            # FORWARD AND BACK PROP
            logits = model(features)
            loss = F.cross_entropy(logits, targets)
            optimizer.zero_grad()

            loss.backward()

            # UPDATE MODEL PARAMETERS
            optimizer.step()

            # LOGGING
            if not batch_idx % 400:
                print('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f'
                      % (epoch+1, num_epochs, batch_idx,
                          len(train_loader), loss))

        with torch.set_grad_enabled(False):
            print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
                  epoch+1, num_epochs,
                  compute_accuracy(model, train_loader, device)))

        print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

    print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))


class CustomLightningModule(L.LightningModule):
    def __init__(self, model, learning_rate=5e-5):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model

        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]  # this is passed to the optimizer for training

    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["label"])
        self.log("val_loss", outputs["loss"], prog_bar=True)

        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.val_acc(predicted_labels, batch["label"])
        self.log("val_acc", self.val_acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["label"])

        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.test_acc(predicted_labels, batch["label"])
        self.log("accuracy", self.test_acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

Fine Tuning Lora

In [ ]:
import os
from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import pandas as pd
import torch

# from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset
# from local_dataset_utilities import IMDBDataset

In [ ]:
if not torch.cuda.is_available():
    print("Please switch to a GPU machine before running this notebook.")


In [ ]:
files = ("test.csv", "train.csv", "val.csv")
download = True

for f in files:
    if not os.path.exists(os.path.join("data", f)):
        download = False


if download is False:
    download_dataset()
    df = load_dataset_into_to_dataframe()
    partition_dataset(df)

In [ ]:
df_train = pd.read_csv(os.path.join("data", "train.csv"))
df_val = pd.read_csv(os.path.join("data", "val.csv"))
df_test = pd.read_csv(os.path.join("data", "test.csv"))

In [ ]:
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": os.path.join("data", "train.csv"),
        "validation": os.path.join("data", "val.csv"),
        "test": os.path.join("data", "test.csv"),
    },
)

print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 10000
    })
})


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [ ]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [ ]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
del imdb_dataset

In [ ]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from torch.utils.data import DataLoader, Dataset


class IMDBDataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows

In [ ]:
train_dataset = IMDBDataset(imdb_tokenized, partition_key="train")
val_dataset = IMDBDataset(imdb_tokenized, partition_key="validation")
test_dataset = IMDBDataset(imdb_tokenized, partition_key="test")

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=12,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=4
)

In [ ]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.W_a = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.W_b = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.W_a @ self.W_b)
        return x


class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

In [ ]:
from functools import partial

lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_query = True
lora_key = False
lora_value = True
lora_projection = False
lora_mlp = False
lora_head = False

layers = []

assign_lora = partial(LinearWithLoRA, rank=lora_r, alpha=lora_alpha)

for layer in model.distilbert.transformer.layer:
    if lora_query:
        layer.attention.q_lin = assign_lora(layer.attention.q_lin)
    if lora_key:
        layer.attention.k_lin = assign_lora(layer.attention.k_lin)
    if lora_value:
        layer.attention.v_lin = assign_lora(layer.attention.v_lin)
    if lora_projection:
        layer.attention.out_lin = assign_lora(layer.attention.out_lin)
    if lora_mlp:
        layer.ffn.lin1 = assign_lora(layer.ffn.lin1)
        layer.ffn.lin2 = assign_lora(layer.ffn.lin2)
if lora_head:
    model.pre_classifier = assign_lora(model.pre_classifier)
    model.classifier = assign_lora(model.classifier)

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (out_lin): Linear(in_features=768, out_features=768, bias

In [ ]:
# Check if linear layers are frozen
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

distilbert.embeddings.word_embeddings.weight: False
distilbert.embeddings.position_embeddings.weight: False
distilbert.embeddings.LayerNorm.weight: False
distilbert.embeddings.LayerNorm.bias: False
distilbert.transformer.layer.0.attention.q_lin.linear.weight: False
distilbert.transformer.layer.0.attention.q_lin.linear.bias: False
distilbert.transformer.layer.0.attention.q_lin.lora.W_a: True
distilbert.transformer.layer.0.attention.q_lin.lora.W_b: True
distilbert.transformer.layer.0.attention.k_lin.weight: False
distilbert.transformer.layer.0.attention.k_lin.bias: False
distilbert.transformer.layer.0.attention.v_lin.linear.weight: False
distilbert.transformer.layer.0.attention.v_lin.linear.bias: False
distilbert.transformer.layer.0.attention.v_lin.lora.W_a: True
distilbert.transformer.layer.0.attention.v_lin.lora.W_b: True
distilbert.transformer.layer.0.attention.out_lin.weight: False
distilbert.transformer.layer.0.attention.out_lin.bias: False
distilbert.transformer.layer.0.sa_layer_no

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 147456


In [ ]:
lightning_model = CustomLightningModule(model)

In [ ]:
callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )  # save top 1 model
]
logger = CSVLogger(save_dir="logs/", name="my-model")

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=10,
)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name     | Type                                | Params | Mode 
-------------------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.1 M | eval 
1 | val_acc  | MulticlassAccuracy                  | 0      | train
2 | test_acc | MulticlassAccuracy                  | 0      | train
-------------------------------------------------------------------------
147 K     Trainable params
67.0 M    Non-trainable params
67.1 M    Total params
268.410   Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name     | Type                                | Params | Mode 
-------------------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.1 M | eval 
1 | val_acc  | MulticlassAccuracy                  | 0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


Time elapsed 17.62 min


In [ ]:
train_acc = trainer.test(lightning_model, dataloaders=train_loader, ckpt_path="best", verbose=False)
val_acc = trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best", verbose=False)
test_acc = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best", verbose=False)

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=1-step=5834.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

In [ ]:
print(f"Train acc: {train_acc[0]['accuracy']*100:2.2f}%")
print(f"Val acc:   {val_acc[0]['accuracy']*100:2.2f}%")
print(f"Test acc:  {test_acc[0]['accuracy']*100:2.2f}%")

Train acc: 93.11%
Val acc:   90.06%
Test acc:  89.62%


Federated Learning

In [ ]:
import shutil

# Cleanup checkpoint files as we don't need them later
log_dir = f"logs/my-model"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

In [ ]:
!pip install flwr

In [ ]:
!pip install flwr[simulation]
!pip install ray

In [ ]:
!pip install flwr
!pip install -U flwr[simulation]


In [ ]:
import flwr as fl
from flwr.common import Metrics
import numpy as np

NUM_CLIENTS = 10

from typing import List
from collections import OrderedDict

def get_parameters(net) -> List[np.ndarray]:
  return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
  params_dict = zip(net.state_dict().keys(), parameters)
  state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
  net.load_state_dict(state_dict, strict = True)

In [ ]:
import flwr as fl
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.trainer = pl.Trainer(max_epochs=1)

    def get_parameters(self):
        return [val.cpu().numpy() for val in self.model.parameters()]

    def set_parameters(self, parameters):
        for param, new_param in zip(self.model.parameters(), parameters):
            param.data = torch.tensor(new_param, dtype=param.dtype)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.trainer.fit(self.model, self.train_loader)
        return self.get_parameters(), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        val_result = self.trainer.test(self.model, self.val_loader, verbose=False)
        val_loss = val_result[0]['test_loss']
        val_acc = val_result[0]['test_acc']  # Assuming your model logs accuracy
        return float(val_loss), len(self.val_loader.dataset), {"accuracy": float(val_acc)}

# Define the model and data loader
model = model
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64)

# Create the Flower client
client = FlowerClient(model, train_loader, val_loader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
def client_fn(cid: str):
    train_loader = train_loader[cid]
    val_loader = val_loader[cid]
    # Assuming `model` is already loaded and prepared with LoRA layers
    return FlowerClient(model, train_loader, val_loader)


In [ ]:
# Define the strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.1,
    fraction_evaluate=0.5,
    min_fit_clients=1,
    min_evaluate_clients=1,
    min_available_clients=1,
)

# Configure the client resources
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_resources = None
if device.type == "cuda":
    client_resources = {"num_gpus": 1}

# Define the model and data loaders
model = model
train_loader_dict = {str(i): DataLoader(dataset=train_dataset, batch_size=64, shuffle=True) for i in range(5)}
val_loader_dict = {str(i): DataLoader(dataset=val_dataset, batch_size=64) for i in range(5)}

client = FlowerClient(model, train_loader_dict, val_loader_dict)

def client_fn(cid: str, train_loader_dict, val_loader_dict):
    return FlowerClient(model, train_loader_dict, val_loader_dict)

# Start the simulation
fl.simulation.start_simulation(
    client_fn=lambda cid: client_fn(cid, train_loader_dict, val_loader_dict),
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
    client_resources=client_resources,
)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-11 14:42:06,532	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 18459129446.0, 'node:__internal_head__': 1.0, 'accelerator_type:L4': 1.0, 'CPU': 16.0, 'node:172.17.0.2': 1.0, 'memory': 36918258894.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
<ipython-input-140-e

RuntimeError: Simulation crashed.

In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): LinearWithLoRA(
              (linear): Linear(in_features=768, out_features=768, bias=True)
              (lora): LoRALayer()
            )
            (out_lin): Linear(in_features=768, out_features=768, bias

In [ ]:
import flwr as fl
from flwr.common import Metrics
import numpy as np
from typing import List
from collections import OrderedDict
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader

# Set up logging
import logging
logging.basicConfig(level=logging.INFO)

NUM_CLIENTS = 1

def get_parameters(self, config):
    return [val.detach().numpy() for _, val in self.model.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.trainer = pl.Trainer(max_epochs=1)

    def get_parameters(self, config):
        return [val.detach().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        for param, new_param in zip(self.model.parameters(), parameters):
            param.data = torch.tensor(new_param, dtype=param.dtype)

    def fit(self, parameters, config):
        logging.info(f"Client starting training with parameters: {parameters}")
        try:
            self.set_parameters(parameters)
            self.trainer.fit(self.model, self.train_loader)
            logging.info("Client finished training")
            return self.get_parameters(), len(self.train_loader.dataset), {}
        except Exception as e:
            logging.error(f"Error during training: {e}")
            raise e

    def evaluate(self, parameters, config):
        logging.info(f"Client starting evaluation with parameters: {parameters}")
        try:
            self.set_parameters(parameters)
            val_result = self.trainer.test(self.model, self.val_loader, verbose=False)
            val_loss = val_result[0]['test_loss']
            val_acc = val_result[0]['test_acc']  # Assuming your model logs accuracy
            logging.info("Client finished evaluation")
            return float(val_loss), len(self.val_loader.dataset), {"accuracy": float(val_acc)}
        except Exception as e:
            logging.error(f"Error during evaluation: {e}")
            raise e

# Define the model and data loaders
model = model
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64)

# Create the Flower client
client = FlowerClient(model, train_loader, val_loader)

def client_fn(cid: str, train_loader_dict, val_loader_dict):
    train_loader = train_loader_dict[cid]
    val_loader = val_loader_dict[cid]
    return FlowerClient(model, train_loader, val_loader).to_client()

# Define the strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.1,
    fraction_evaluate=0.5,
    min_fit_clients=1,
    min_evaluate_clients=1,
    min_available_clients=1,
)

# Configure the client resources
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_resources = {"num_gpus": 1} if device.type == "cuda" else None

# Define the model and data loaders
model = model
train_loader_dict = {str(i): DataLoader(dataset=train_dataset, batch_size=64, shuffle=True) for i in range(5)}
val_loader_dict = {str(i): DataLoader(dataset=val_dataset, batch_size=64) for i in range(5)}

# Start the simulation
logging.info("Starting Flower simulation")
print("Starting Flower simulation")
try:
    fl.simulation.start_simulation(
        client_fn=lambda cid: client_fn(cid, train_loader_dict, val_loader_dict),
        num_clients=5,
        config=fl.server.ServerConfig(num_rounds=5),
        strategy=strategy,
        client_resources=client_resources,
    )
except Exception as e:
    print(f"Error during simulation: {e}")
    logging.error(f"Error during simulation: {e}")


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Starting Flower simulation


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-11 17:58:13,111	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:L4': 1.0, 'node:__internal_head__': 1.0, 'memory': 35234719335.0, 'CPU': 16.0, 'node:172.17.0.2': 1.0, 'object_store_memory': 17617359667.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
<ipython-input-147-a7619683e745>:97: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_cpus=1` for each client.
  fl.simulation.start_simulation(
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :     

KeyboardInterrupt: 

In [ ]:
import flwr as fl
from flwr.common import Metrics
import numpy as np
from typing import List
from collections import OrderedDict
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import logging

logging.basicConfig(level=logging.INFO)

NUM_CLIENTS = 10

def get_parameters(self, config):
    return [val.detach().numpy() for _, val in self.model.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.trainer = pl.Trainer(max_epochs=1)

    def get_parameters(self, config):
        return [val.detach().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        for param, new_param in zip(self.model.parameters(), parameters):
            param.data = torch.tensor(new_param, dtype=param.dtype)

    def fit(self, parameters, config):
        logging.info(f"Client starting training with parameters: {parameters}")
        try:
            self.set_parameters(parameters)
            self.trainer.fit(self.model, self.train_loader)
            logging.info("Client finished training")
            return self.get_parameters({}), len(self.train_loader.dataset), {}
        except Exception as e:
            logging.error(f"Error during training: {e}")
            raise e

    def evaluate(self, parameters, config):
        logging.info(f"Client starting evaluation with parameters: {parameters}")
        try:
            self.set_parameters(parameters)
            val_result = self.trainer.test(self.model, self.val_loader, verbose=False)
            val_loss = val_result[0]['test_loss']
            val_acc = val_result[0]['test_acc']  # Assuming your model logs accuracy
            logging.info("Client finished evaluation")
            return float(val_loss), len(self.val_loader.dataset), {"accuracy": float(val_acc)}
        except Exception as e:
            logging.error(f"Error during evaluation: {e}")
            raise e

# Define the model and data loaders
# Assuming 'model', 'train_dataset', and 'val_dataset' are defined somewhere
train_loader_dict = {str(i): DataLoader(dataset=train_dataset, batch_size=64, shuffle=True) for i in range(NUM_CLIENTS)}
val_loader_dict = {str(i): DataLoader(dataset=val_dataset, batch_size=64) for i in range(NUM_CLIENTS)}

def client_fn(cid: str):
    logging.info(f"Creating client {cid}")
    train_loader = train_loader_dict[cid]
    val_loader = val_loader_dict[cid]
    return FlowerClient(model, train_loader, val_loader)

# Define the strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.1,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
)

# Configure the client resources
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_resources = {"num_gpus": 1} if device.type == "cuda" else None

# Start the simulation
logging.info("Starting Flower simulation")
print("Starting Flower simulation")
try:
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=5),
        strategy=strategy,
        client_resources=client_resources,
    )
except Exception as e:
    print(f"Error during simulation: {e}")
    logging.error(f"Error during simulation: {e}")


INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout


Starting Flower simulation


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-11 17:36:27,838	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 17837877657.0, 'node:172.17.0.2': 1.0, 'CPU': 16.0, 'memory': 35675755316.0, 'GPU': 1.0, 'accelerator_type:L4': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
<ipython-input-144-7a17a87dbae6>:89: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_cpus=1` for each client.
  fl.simulation.start_simulation(
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 1, 'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :     

KeyboardInterrupt: 

In [ ]:
from typing import List, Tuple

def weighted_average(metrics : List[Tuple[int, Metrics]]) -> Metrics:
  accuracies = [num_examples * metric["accuracy"] for num_examples, metric in metrics]
  examples = [num_examples for num_examples, _ in metrics]
  return {"accuracy": sum(accuracies) / sum(examples)}